In [1]:
from setup_general import *
from setup_embedding import *

/home/till/projects/uni/data science/museum_item_classification/setup_general.py:92: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  combined_intermediate_ready = pd.read_csv('./data/general/combined_intermediate_ready.csv', index_col='id', dtype={'type': str})


# Preparation - extract text from ds

In [ ]:
def collect_text(item):
    return ' '.join(item[text_features]).strip()

data = prep.copy()

text_features = ['name', 'commentary', 'text', 'legend', 'initial_info', 'additional_text']
data[text_features] = data[text_features].fillna('')
data['text_features'] = data.apply(lambda item: collect_text(item),axis=1)

data.to_csv('data/prep.csv')

In [ ]:
data = prep.copy()
with_damages = combined_data_fully_translated.copy()

data.text_features = data.text_features.replace(float('nan'), ' ',)
with_damages.damages = with_damages.damages.replace(float('nan'), ' ',)

data.text_features = data.text_features + ' ' + with_damages.damages

data.to_csv('data/prep.csv')

In [ ]:
data = prep.copy()
text = data[['text_features','type','source']]
text.to_csv('data/text.csv')

In [ ]:
data = text.copy()
data.text_features = data.text_features.apply(lambda x: x.strip())
data = data[data.text_features != '']
data.to_csv('data/text.csv')

In [ ]:
df = text.copy()
train = df[df.source == 'train']
test = df[df.source == 'test']
train.to_csv('data/train_text.csv', index=True)
test.to_csv('data/test_text.csv', index=True)

# extract in Estonian

In [6]:
def collect_text(item):
    return ' '.join(item[text_features]).strip()

data = combined_data.copy()

text_features = ['name', 'commentary', 'text', 'legend', 'initial_info', 'additional_text', 'damages']
data[text_features] = data[text_features].fillna('')
data['text_features'] = data.apply(lambda item: collect_text(item),axis=1)

text = data[['text_features','type','source']]

text.text_features = text.text_features.apply(lambda x: x.strip())
text = text[text.text_features != '']

train = text[text.source == 'train'].drop('source', axis=1)
test = text[text.source == 'test'].drop('source', axis=1)
train.to_csv('data/text/train_text_est.csv', index=True)
test.to_csv('data/text/test_text_est.csv', index=True)
text.to_csv('data/text/text_est.csv')

/tmp/ipykernel_36267/1302915245.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  text.text_features = text.text_features.apply(lambda x: x.strip())


# gpt3 text embeddings


In [ ]:
openai.api_key = '***'
count = 0
def get_embedding(text, model="text-similarity-davinci-001"):
    global count
    count += 1
    text = text.replace("\n", " ")
    try:
        result = openai.Embedding.create(input = [text], model=model)['data'][0]['embedding']
    except:
        print(count)
        time.sleep(60)
        result = openai.Embedding.create(input = [text], model=model)['data'][0]['embedding']
    return result
 
text['curie_similarity'] = text.text_features.apply(lambda x: get_embedding(x, model='text-similarity-curie-001'))
text.to_csv('data/text_embeddings/curie.csv', index=False)

In [ ]:
data = curie.copy()
# remove unneeded cols

data.drop(columns=['text_features'], inplace=True)
data.set_index('id', inplace=True)
split = pd.DataFrame(data.curie_similarity.tolist(), index=data.index)
concat = pd.concat([data, split], axis=1)
concat.drop(columns=['curie_similarity'], inplace=True)
concat.to_csv('data/text_embeddings/curie.csv')

# train val split - embedding


In [ ]:
train = train_est_prepared.copy()
val = val_est_prepared.copy()
trainval_curie = curie[curie.source == 'train'].drop(columns=['source'])
test_curie = curie[curie.source == 'test'].drop(columns=['source'])

train_curie = pd.DataFrame.join(train[['element_count']], trainval_curie)
train_curie.dropna(axis=0, inplace=True)
train_curie.drop(columns=['element_count'], inplace=True)
print(len(train_curie))
val_curie = pd.DataFrame.join(val[['element_count']], trainval_curie)
val_curie.dropna(axis=0, inplace=True)
val_curie.drop(columns=['element_count'], inplace=True)
print(len(val_curie))

train_curie.to_csv('data/text_embeddings/train_curie.csv', index_label='id')
val_curie.to_csv('data/text_embeddings/val_curie.csv', index_label='id')
test_curie.to_csv('data/text_embeddings/test_curie.csv', index_label='id')

# bag of words & train val split


In [2]:
min_gram = 1
max_gram = 2
max_features = 4000

dataset = text_est
stop_words = stopwords_est

CountVec = TfidfVectorizer(ngram_range=(min_gram,max_gram), stop_words=stop_words, max_features=max_features)
# to use bigrams ngram_range=(2,2)
Count_data = CountVec.fit_transform(dataset.text_features)

#create dataframe
bow=pd.DataFrame(Count_data.toarray(),columns=CountVec.get_feature_names())

bow = bow.add_prefix('word_')
bow.index = dataset.index
bow = bow.join(dataset[['source', 'type']])

train = train_est_prepared.copy()
val = val_est_prepared.copy()
trainval_bow = bow[bow.source == 'train'].drop(columns=['source'])
test_bow = bow[bow.source == 'test'].drop(columns=['source'])

train_bow = pd.DataFrame.join(train[['element_count']], trainval_bow)
train_bow.dropna(axis=0, inplace=True)
train_bow.drop(columns=['element_count'], inplace=True)
print(len(train_bow))
val_bow = pd.DataFrame.join(val[['element_count']], trainval_bow)
val_bow.dropna(axis=0, inplace=True)
val_bow.drop(columns=['element_count'], inplace=True)
print(len(val_bow))

train_bow.to_csv('data/text_embeddings/train_bow.csv', index_label='id')
val_bow.to_csv('data/text_embeddings/val_bow.csv', index_label='id')
test_bow.to_csv('data/text_embeddings/test_bow.csv', index_label='id')

/home/till/.local/lib/python3.10/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


7284
3166
